In [1]:
import pandas as pd
import numpy as np 
import pickle
import pickle5
import matplotlib.pyplot as plt
import datetime
from pysurvival.models.semi_parametric import NonLinearCoxPHModel
from pysurvival.utils.display import display_loss_values
from sklearn.preprocessing import StandardScaler
from pysurvival.utils.display import compare_to_actual
from pysurvival.utils import save_model, load_model
from pysurvival.utils.sklearn_adapter import sklearn_adapter
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.metrics import auc, r2_score
from sklearn import metrics
from sklearn.utils import resample
from pysurvival.utils.metrics import concordance_index
from lifelines.utils import concordance_index as c_index
## block warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
with open("../data/ut_V6_classification-4H.pkl", "rb") as fh:
    df = pickle5.load(fh)
    
df.head(3)

,life_time,datetime_start,datetime_end,latitude,longitude,distance_center_km,weekday_b,charging_ports,lag1,lag2,...,hour_b_22,hour_b_23,Station_Name_BOULDER / N BOULDER REC 1,Station_Name_COMM VITALITY / 1000WALNUT,Station_Name_COMM VITALITY / 1104 SPRUCE1,4Hsplit,y_pred,Station_Name,weekday_b_name,tod
3734,118.5,2018-01-05 14:25:00,2018-01-05 16:23:30,40.000148,-105.282437,2.096847,4,2,1271.5,135.5,...,0,0,0,0,0,0,1,BOULDER / BASELINE ST1,Friday,Midday
3735,886.5,2018-01-05 17:02:00,2018-01-06 07:48:30,40.000148,-105.282437,2.096847,4,2,118.5,1271.5,...,0,0,0,0,0,1,1,BOULDER / BASELINE ST1,Friday,Afternoon
3736,326.5,2018-01-06 09:28:00,2018-01-06 14:54:30,40.000148,-105.282437,2.096847,5,2,886.5,118.5,...,0,0,0,0,0,1,0,BOULDER / BASELINE ST1,Saturday,Morning


## Modeling features

To make coding easier the names of the models are changed:
- M1: Baseline
- M5: Full

In [28]:
df['event'] = np.ones(len(df))

In [29]:
time_column = 'life_time'
event_column = 'event'

## M1
features1 = ['weekday_b_name_Monday', 'weekday_b_name_Saturday',
             'weekday_b_name_Sunday', 'weekday_b_name_Thursday',
             'weekday_b_name_Tuesday', 'weekday_b_name_Wednesday', 'tod_Evening', 'tod_Midday', 'tod_Morning',
       'Station_Name_BOULDER / N BOULDER REC 1',
       'Station_Name_COMM VITALITY / 1000WALNUT',
       'Station_Name_COMM VITALITY / 1104 SPRUCE1'] # dow + tod

## M5
features5_cat = ['weekday_b_name_Monday', 'weekday_b_name_Saturday',
             'weekday_b_name_Sunday', 'weekday_b_name_Thursday',
             'weekday_b_name_Tuesday', 'weekday_b_name_Wednesday','tod_Evening', 'tod_Midday', 'tod_Morning',
       'Station_Name_BOULDER / N BOULDER REC 1',
       'Station_Name_COMM VITALITY / 1000WALNUT',
       'Station_Name_COMM VITALITY / 1104 SPRUCE1']
features5_con = ['lag1', 'lag2', 'lag3','near_charge_time_4H', 'near_charge_energy_4H',
                 'charge_time_4H', 'charge_energy_4H','service', 'entertainment', 'food', 
                 'childcare', 'medical', 'education', 'waste-management']

In [30]:
#data split
split = 0.8

In [31]:
train, test = np.split(df, [int(split * len(df))])
print("Training shape:",train.shape)
print("Testing shape:", test.shape)

Training shape: (4088, 103)
Testing shape: (1022, 103)


In [32]:
trainS = train[train['y_pred'] == 0]
trainL = train[train['y_pred'] == 1]

testS = test[test['y_pred'] == 0]
testL = test[test['y_pred'] == 1]

In [33]:
print("SHORT: Train: {} Test: {}".format(trainS.shape[0], testS.shape[0]))
print("LONG: Train: {} Test: {}".format(trainL.shape[0], testL.shape[0]))

SHORT: Train: 2433 Test: 629
LONG: Train: 1655 Test: 393


In [34]:
## Get X 
# Baseline
X1_S = trainS[features1]
X1_L = trainL[features1]

## Full
# Short
X5_S = trainS[features5_cat+features5_con]
scaler = StandardScaler()
X5_S[features5_con] = scaler.fit_transform(X5_S[features5_con])
X5_S_test = testS[features5_cat+features5_con]
X5_S_test[features5_con] = scaler.transform(X5_S_test[features5_con])
# Full
X5_L = trainL[features5_cat+features5_con]
scaler = StandardScaler()
X5_L[features5_con] = scaler.fit_transform(X5_L[features5_con])
X5_L_test = testL[features5_cat+features5_con]
X5_L_test[features5_con] = scaler.transform(X5_L_test[features5_con])

## Get E -train
E_S = trainS[event_column]
E_L = trainL[event_column]

## Get T - train
T_S = trainS['life_time']
T_L = trainL['life_time']

## Get E -Test
E_S_test = testS[event_column]
E_L_test = testL[event_column]

## Get T - Test
T_S_test = testS['life_time']
T_L_test = testL['life_time']

In [36]:
## define grid for RandomizedsearchCV
## One layers
struc10 = [{'activation': 'ReLU', 'num_units': 16}]
struc11 = [{'activation': 'ReLU', 'num_units': 32}]
struc12 = [{'activation': 'ReLU', 'num_units': 64}]
struc13 = [{'activation': 'ReLU', 'num_units': 128}]
struc14 = [{'activation': 'ReLU', 'num_units': 264}]

## Two layers
struc20 = [{'activation': 'ReLU', 'num_units': 16},{'activation': 'ReLU', 'num_units': 16}]
struc21 = [{'activation': 'ReLU', 'num_units': 32},{'activation': 'ReLU', 'num_units': 32}]
struc22 = [{'activation': 'ReLU', 'num_units': 64},{'activation': 'ReLU', 'num_units': 64}]
struc23 = [{'activation': 'ReLU', 'num_units': 128},{'activation': 'ReLU', 'num_units': 128}]
struc24 = [{'activation': 'ReLU', 'num_units': 264},{'activation': 'ReLU', 'num_units': 264}]

## Three layers
struc30 = [{'activation': 'ReLU', 'num_units': 16},{'activation': 'ReLU', 'num_units': 16},
           {'activation': 'ReLU', 'num_units': 16}]
struc31 = [{'activation': 'ReLU', 'num_units': 32},{'activation': 'ReLU', 'num_units': 32},
           {'activation': 'ReLU', 'num_units': 32}]
struc32 = [{'activation': 'ReLU', 'num_units': 64},{'activation': 'ReLU', 'num_units': 64}, 
           {'activation': 'ReLU', 'num_units': 64}]
struc33 = [{'activation': 'ReLU', 'num_units': 128},{'activation': 'ReLU', 'num_units': 128},
           {'activation': 'ReLU', 'num_units': 128}]
struc34 = [{'activation': 'ReLU', 'num_units': 264},{'activation': 'ReLU', 'num_units': 264},
           {'activation': 'ReLU', 'num_units': 264}]

## Four layers
struc40 = [{'activation': 'ReLU', 'num_units': 16},{'activation': 'ReLU', 'num_units': 16},
           {'activation': 'ReLU', 'num_units': 16},{'activation': 'ReLU', 'num_units': 16}]
struc41 = [{'activation': 'ReLU', 'num_units': 32},{'activation': 'ReLU', 'num_units': 32},
           {'activation': 'ReLU', 'num_units': 32},{'activation': 'ReLU', 'num_units': 32}]
struc42 = [{'activation': 'ReLU', 'num_units': 64},{'activation': 'ReLU', 'num_units': 64}, 
           {'activation': 'ReLU', 'num_units': 64},{'activation': 'ReLU', 'num_units': 64}]
struc43 = [{'activation': 'ReLU', 'num_units': 128},{'activation': 'ReLU', 'num_units': 128},
           {'activation': 'ReLU', 'num_units': 128},{'activation': 'ReLU', 'num_units': 128}]
struc44 = [{'activation': 'ReLU', 'num_units': 264},{'activation': 'ReLU', 'num_units': 264},
           {'activation': 'ReLU', 'num_units': 264},{'activation': 'ReLU', 'num_units': 264}]

## define grid
grid = {'lr':[0.0001,0.001,0.01,0.1], 'l2_reg':[0.0001,0.001,0.01,1,10],
        'structure':[struc10,struc11,struc12,struc13,struc14,struc20,struc21,struc22,struc23,struc24,
                     struc30,struc31,struc32,struc33,struc34,struc40,struc41,struc42,struc43,struc44],
        'dropout':[0.1,0.25,0.5], 'num_epochs':[10,100,500,1000]}

In [37]:
results = {}

### Find hyper parameters
Short:

In [14]:
########### M1
#### SHORT
## sklearn adapter
dsSklearn = sklearn_adapter(NonLinearCoxPHModel, time_col=time_column, event_col=event_column,
                          predict_method="predict_survival", scoring_method=concordance_index)
dsSkl = dsSklearn()
reg = RandomizedSearchCV(dsSkl, grid, random_state=42, n_iter=50, cv=5) # if n_iter = 100 it will train 100 models.
reg.fit(X1_S,y_S, verbose=False, batch_normalization=False)
paramS1 = reg.best_params_
print("Parameters, baseline - SHORT",paramS1)

#### LONG
## sklearn adapter
dsSklearn = sklearn_adapter(NonLinearCoxPHModel, time_col=time_column, event_col=event_column,
                          predict_method="predict_survival", scoring_method=concordance_index)
dsSkl = dsSklearn()
reg = RandomizedSearchCV(dsSkl, grid, random_state=42, n_iter=50, cv=5) # if n_iter = 100 it will train 100 models.
reg.fit(X1_L,y_L, verbose=False, batch_normalization=False)
paramL1 = reg.best_params_
print("Parameters, baseline - LONG",paramL1)

########### M5
#### SHORT
## sklearn adapter
dsSklearn = sklearn_adapter(NonLinearCoxPHModel, time_col=time_column, event_col=event_column,
                          predict_method="predict_survival", scoring_method=concordance_index)
dsSkl = dsSklearn()
reg = RandomizedSearchCV(dsSkl, grid, random_state=42, n_iter=50, cv=5) # if n_iter = 100 it will train 100 models.
reg.fit(X5_S,y_S, verbose=False, batch_normalization=False)
paramS5 = reg.best_params_
print("Parameters, full - SHORT",paramS5)

#### LONG
## sklearn adapter
dsSklearn = sklearn_adapter(NonLinearCoxPHModel, time_col=time_column, event_col=event_column,
                          predict_method="predict_survival", scoring_method=concordance_index)
dsSkl = dsSklearn()
reg = RandomizedSearchCV(dsSkl, grid, random_state=42, n_iter=50, cv=5) # if n_iter = 100 it will train 100 models.
reg.fit(X5_L,y_L, verbose=False, batch_normalization=False)
paramL5 = reg.best_params_
print("Parameters, full - LONG",paramL5)

Parameters, baseline - SHORT {'structure': [{'activation': 'ReLU', 'num_units': 32}], 'num_epochs': 10, 'lr': 0.1, 'l2_reg': 10, 'dropout': 0.25}
Parameters, baseline - LONG {'structure': [{'activation': 'ReLU', 'num_units': 128}, {'activation': 'ReLU', 'num_units': 128}], 'num_epochs': 100, 'lr': 0.01, 'l2_reg': 0.01, 'dropout': 0.1}
Parameters, full - SHORT {'structure': [{'activation': 'ReLU', 'num_units': 16}], 'num_epochs': 1000, 'lr': 0.0001, 'l2_reg': 1, 'dropout': 0.1}
Parameters, full - LONG {'structure': [{'activation': 'ReLU', 'num_units': 128}, {'activation': 'ReLU', 'num_units': 128}], 'num_epochs': 100, 'lr': 0.01, 'l2_reg': 0.01, 'dropout': 0.1}


In [ ]:
## To test model
def test_model(y_test, y_pred):
    MAE = metrics.mean_absolute_error(y_test, y_pred)
    MSE = metrics.mean_squared_error(y_test, y_pred)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    MAPE = mean_absolute_percentage_error(y_test, y_pred)
    NRMSE = RMSE/np.mean(y_test)
    return MAE, RMSE, MSE, MAPE, NRMSE

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))*100

#### DeepSurv - Model 1

In [60]:
#########################################################
#################### SEPERATE RESULTS ###################

results['M1'] = {}
results['M1']['S'] = {}
results['M1']['L'] = {}
results['M1']['Com'] = {}

################ Fit models ################
#
####################### short #######################
print("SHORT")
## Fit model
reg = NonLinearCoxPHModel(structure=paramS1['structure'])
reg.fit(X1_S, T_S, E_S, lr=paramS1['lr'], l2_reg=paramS1['l2_reg'], 
        batch_normalization=False, verbose=False, num_epochs=paramS1['num_epochs'],
        dropout=paramS1['dropout'])

## Point predictions
T_test_pred_S, T_train_pred_S = point_pred(reg, X1_S_test, X1_S)

## Test model
c_test = c_index(T_S_test, T_test_pred_S)
c_train = c_index(T_S, T_train_pred_S)
print("Model tested")

## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(T_S_test,T_test_pred_S)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(T_S,T_train_pred_S)

## Save results
results['M1']['S'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                  'MAE_train': MAE_train,'MAE_test':MAE_test,
                  'R2_test':r2_score(T_S_test,T_test_pred_S), 
                  'R2_train':r2_score(T_S,T_train_pred_S),
                  'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                  'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train,
                  'c-train':c_train,'c-test':c_test}

####################### long #######################
print("LONG")

## Fit model
reg = NonLinearCoxPHModel(structure=paramL1['structure'])
reg.fit(X1_L, T_L, E_L, lr=paramL1['lr'], l2_reg=paramL1['l2_reg'], 
        batch_normalization=False, verbose=False, num_epochs=paramL1['num_epochs'],
        dropout=paramL1['dropout'])

## Point predictions
T_test_pred_L, T_train_pred_L = point_pred(reg, X1_L_test, X1_L)

## Test model
c_test = c_index(T_L_test, T_test_pred_L)
c_train = c_index(T_L, T_train_pred_L)
print("Model tested")

## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(T_L_test,T_test_pred_L)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(T_L,T_train_pred_L)

## Save results
results['M1']['L'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                  'MAE_train': MAE_train,'MAE_test':MAE_test,
                  'R2_test':r2_score(T_L_test,T_test_pred_L), 
                  'R2_train':r2_score(T_L,T_train_pred_L),
                  'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                  'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train,
                  'c-train':c_train,'c-test':c_test}

###############################################################
############ COMBINED ###################

T_com_real_test1 = pd.concat([T_S_test, T_L_test])
T_test_pred_S.extend(T_test_pred_L)
T_com_pred_test1 = T_test_pred_S

T_com_real_train = pd.concat([T_S, T_L])
T_train_pred_S.extend(T_train_pred_L)
T_com_pred_train = T_train_pred_S

## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(T_com_real_test1, T_com_pred_test1)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(T_com_real_train, T_com_pred_train) 

## C-index
c_test = c_index(T_com_real_test1, T_com_pred_test1)
c_train = c_index(T_com_real_train, T_com_pred_train)

## Save results
results['M1']['Com'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                      'MAE_train': MAE_train,'MAE_test':MAE_test,
                      'R2_test':r2_score(T_com_real_test1,T_com_pred_test1), 
                      'R2_train':r2_score(T_com_real_train,T_com_pred_train),
                      'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                      'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train,
                      'c-train':c_train,'c-test':c_test}

SHORT
Model tested
LONG
Model tested


#### DeepSurv - Model 5

In [62]:
#########################################################
#################### SEPERATE RESULTS ###################

results['M5'] = {}
results['M5']['S'] = {}
results['M5']['L'] = {}
results['M5']['Com'] = {}

################ Fit models ################
#
####################### short #######################
print("SHORT")
## Fit model
reg = NonLinearCoxPHModel(structure=paramS5['structure'])
reg.fit(X5_S, T_S, E_S, lr=paramS5['lr'], l2_reg=paramS5['l2_reg'], 
        batch_normalization=False, verbose=False, num_epochs=paramS5['num_epochs'],
        dropout=paramS5['dropout'])

## Point predictions
T_test_pred_S, T_train_pred_S = point_pred(reg, X5_S_test, X5_S)

## Test model
c_test = c_index(T_S_test, T_test_pred_S)
c_train = c_index(T_S, T_train_pred_S)
print("Model tested")

## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(T_S_test,T_test_pred_S)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(T_S,T_train_pred_S)

## Save results
results['M5']['S'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                  'MAE_train': MAE_train,'MAE_test':MAE_test,
                  'R2_test':r2_score(T_S_test,T_test_pred_S), 
                  'R2_train':r2_score(T_S,T_train_pred_S),
                  'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                  'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train,
                  'c-train':c_train,'c-test':c_test}

####################### long #######################
print("LONG")

## Fit model
reg = NonLinearCoxPHModel(structure=paramL5['structure'])
reg.fit(X5_L, T_L, E_L, lr=paramL5['lr'], l2_reg=paramL5['l2_reg'], 
        batch_normalization=False, verbose=False, num_epochs=paramL5['num_epochs'],
        dropout=paramL1['dropout'])

## Point predictions
T_test_pred_L, T_train_pred_L = point_pred(reg, X5_L_test, X5_L)

## Test model
c_test = c_index(T_L_test, T_test_pred_L)
c_train = c_index(T_L, T_train_pred_L)
print("Model tested")

## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(T_L_test,T_test_pred_L)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(T_L,T_train_pred_L)

## Save results
results['M5']['L'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                  'MAE_train': MAE_train,'MAE_test':MAE_test,
                  'R2_test':r2_score(T_L_test,T_test_pred_L), 
                  'R2_train':r2_score(T_L,T_train_pred_L),
                  'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                  'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train,
                  'c-train':c_train,'c-test':c_test}

###############################################################
############ COMBINED ###################

T_com_real_test5 = pd.concat([T_S_test, T_L_test])
T_test_pred_S.extend(T_test_pred_L)
T_com_pred_test5 = T_test_pred_S

T_com_real_train = pd.concat([T_S, T_L])
T_train_pred_S.extend(T_train_pred_L)
T_com_pred_train = T_train_pred_S

## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(T_com_real_test5, T_com_pred_test5)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(T_com_real_train, T_com_pred_train) 

## C-index
c_test = c_index(T_com_real_test5, T_com_pred_test5)
c_train = c_index(T_com_real_train, T_com_pred_train)

## Save results
results['M5']['Com'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                      'MAE_train': MAE_train,'MAE_test':MAE_test,
                      'R2_test':r2_score(T_com_real_test5,T_com_pred_test5), 
                      'R2_train':r2_score(T_com_real_train,T_com_pred_train),
                      'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                      'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train,
                      'c-train':c_train,'c-test':c_test}

SHORT
Model tested
LONG
Model tested


In [65]:
## save results dictionary
with open("results/DeepSurv_predictions_twostage.pickle", 'wb') as handle:
    pickle.dump(combined_pred, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Results

In [80]:
models = ['M1', 'M5']

print("SHORT")
for mo in models:
    r = results[mo]['S']   
    RMSE = r['RMSE_test']
    R2 = r['R2_test']
    MAE = r['MAE_test']
    MAPE = r['MAPE_test']
    NRMSE = r['NRMSE_test']
    C_idx = r['c-test']

    print("DeepSurv {}: R2: {} RMSE: {} C-idx: {} MAE: {} MAPE: {} NRMSE: {}".format(mo,
                                                                          round(R2,2),
                                                                          round(RMSE,0),
                                                                          round(C_idx,2),
                                                                          round(MAE,0),
                                                                          round(MAPE,0),
                                                                          round(NRMSE,2))) 

SHORT
DeepSurv M1: R2: -0.02 RMSE: 403.0 C-idx: 0.52 MAE: 317.0 MAPE: 2047.0 NRMSE: 1.43
DeepSurv M5: R2: -0.05 RMSE: 409.0 C-idx: 0.53 MAE: 330.0 MAPE: 2252.0 NRMSE: 1.45


In [79]:
models = ['M1', 'M5']

print("LONG")
for mo in models:
    r = results[mo]['L']   
    RMSE = r['RMSE_test']
    R2 = r['R2_test']
    MAE = r['MAE_test']
    MAPE = r['MAPE_test']
    NRMSE = r['NRMSE_test']
    C_idx = r['c-test']

    print("DeepSurv {}: R2: {} RMSE: {} C-idx: {} MAE: {} MAPE: {} NRMSE: {}".format(mo,
                                                                          round(R2,2),
                                                                          round(RMSE,0),
                                                                          round(C_idx,2),
                                                                          round(MAE,0),
                                                                          round(MAPE,0),
                                                                          round(NRMSE,2))) 

LONG
DeepSurv M1: R2: -0.02 RMSE: 458.0 C-idx: 0.6 MAE: 343.0 MAPE: 1340.0 NRMSE: 0.61
DeepSurv M5: R2: -0.17 RMSE: 489.0 C-idx: 0.6 MAE: 365.0 MAPE: 1469.0 NRMSE: 0.65


In [78]:
models = ['M1', 'M5']

print("Combined")
for mo in models:
    r = results[mo]['Com']   
    RMSE = r['RMSE_test']
    R2 = r['R2_test']
    MAE = r['MAE_test']
    MAPE = r['MAPE_test']
    NRMSE = r['NRMSE_test']
    C_idx = r['c-test']

    print("DeepSurv {}: R2: {} RMSE: {} C-idx: {} MAE: {} MAPE: {} NRMSE: {}".format(mo,
                                                                          round(R2,2),
                                                                          round(RMSE,0),
                                                                          round(C_idx,2),
                                                                          round(MAE,0),
                                                                          round(MAPE,0),
                                                                          round(NRMSE,2))) 

Combined
DeepSurv M1: R2: 0.21 RMSE: 425.0 C-idx: 0.64 MAE: 327.0 MAPE: 1775.0 NRMSE: 0.92
DeepSurv M5: R2: 0.15 RMSE: 441.0 C-idx: 0.63 MAE: 344.0 MAPE: 1951.0 NRMSE: 0.96
